In [1]:
from modis_tools.auth import ModisSession
from modis_tools.resources import CollectionApi, GranuleApi
from modis_tools.granule_handler import GranuleHandler

from datetime import datetime, timedelta
from pathlib import Path

### Use modis-tools to search and download MODIS granules

In [2]:
username = "flojogreaves"
password = ".V.F3L+-gGA67Sp"

# Authenticate a session
session = ModisSession(username=username, password=password)

In [3]:
# Query the MODIS catalog for collections
collection_client = CollectionApi(session=session)
collections = collection_client.query(short_name="MYD02HKM", version="6.1")
collections

[Collection(id='C1379758778-LAADS', title='MODIS/Aqua Calibrated Radiances 5-Min L1B Swath 500m', dataset_id='MODIS/Aqua Calibrated Radiances 5-Min L1B Swath 500m', coordinate_system='CARTESIAN', time_start='2002-07-04T00:45:00.000Z', updated=None, links=[CollectionLink(hreflang='en-US', href=AnyUrl('https://doi.org/10.5067/MODIS/MYD02HKM.061', scheme='https', host='doi.org', tld='org', host_type='domain', path='/10.5067/MODIS/MYD02HKM.061'), type=None), CollectionLink(hreflang='en-US', href=AnyUrl('https://mcst.gsfc.nasa.gov/sites/default/files/file_attachments/M1054D_PUG_083112_final.pdf', scheme='https', host='mcst.gsfc.nasa.gov', tld='gov', host_type='domain', path='/sites/default/files/file_attachments/M1054D_PUG_083112_final.pdf'), type=None), CollectionLink(hreflang='en-US', href=AnyUrl('https://mcst.gsfc.nasa.gov/sites/mcst.gsfc/files/file_attachments/MODIS_L1B_ATBD_ver4.pdf', scheme='https', host='mcst.gsfc.nasa.gov', tld='gov', host_type='domain', path='/sites/mcst.gsfc/files

In [4]:
# Query the selected collection for granules
granule_client = GranuleApi.from_collection(collections[0], session=session)

In [5]:
# Filter the selected granules via spatial and temporal parameters
# e.g.
# nigeria_bbox = [2.1448863675, 4.002583177, 15.289420717, 14.275061098]
# nigeria_granules = granule_client.query(start_date="2016-01-01", end_date="2018-12-31", bounding_box=nigeria_bbox)

# See https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html for more options

# Date range:
start_date = "2022-01-01"
end_date = "2022-12-31"

# Spatial extent:
labsea = [-65.0, 45.0, -40.0, 70.0] # bounding_box for Labrador Sea
points = ["-55.0,53.75", "-64.25,61.5", "-53.25,64.25", "-44.0,59.25"] # four approximate 'corners' of the Labrador Sea
points2 = ["-56.0,52.5", "-68.0,60.0", "-53.0,66.0", "-41.0,60.0"] # four corners of a larger area
points3 = ["-64.0,55.0", "-44.0,63.0"] # two ends of a line bisecting the Labrador Sea

labsea2022_granules = granule_client.query(start_date=start_date, end_date=end_date, day_night_flag="day", 
                                         point=points2) 
                                            # point co-ordinate is in lon, lat (opposite to earthdata web search!)

# kwargs must be passed as dict if any options are to be used, e.g OR instead of AND

# labsea2022_granules = granule_client.query(start_date=start_date, end_date=end_date, 
#                                           **{"point":["-52.5,57.5", "-62.4,57.5"], "options[point][or]":"true",
#                                              "day_night_flag":"day"})



In [6]:
# Check how many granules before downloading!
labsea2022_granules = list(labsea2022_granules)
len(labsea2022_granules)

71

In [7]:
labsea2022_granules[0:10]

[Granule(id='G2568663137-LAADS', title='LAADS:7257808017', dataset_id='MODIS/Aqua Calibrated Radiances 5-Min L1B Swath 500m', coordinate_system='GEODETIC', time_start='2022-12-18T16:40:00.000Z', updated=datetime.datetime(2022, 12, 21, 3, 25, 42, 870000, tzinfo=datetime.timezone.utc), links=[GranuleLink(hreflang='en-US', href=AnyUrl('https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/61/MYD02HKM/2022/352/MYD02HKM.A2022352.1640.061.2022354222030.hdf', scheme='https', host='ladsweb.modaps.eosdis.nasa.gov', tld='gov', host_type='domain', path='/archive/allData/61/MYD02HKM/2022/352/MYD02HKM.A2022352.1640.061.2022354222030.hdf'), type='application/x-hdfeos', inherited=None), GranuleLink(hreflang='en-US', href=AnyUrl('s3://prod-lads/MYD02HKM/MYD02HKM.A2022352.1640.061.2022354222030.hdf', scheme='s3', host='prod-lads', host_type='int_domain', path='/MYD02HKM/MYD02HKM.A2022352.1640.061.2022354222030.hdf'), type='application/x-hdfeos', inherited=None), GranuleLink(hreflang='en-US', href=AnyU

In [8]:
# Specify download folder

folder = "AquaHkmLabSea2022b"
filepath = Path("/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/"+folder)
filepath.mkdir(exist_ok=True, parents=True)

In [9]:
# Save the list of granules to a text file
with open(filepath / "granule_list.txt", "w") as f:
    for granule in labsea2022_granules:
        f.write(f"{granule}\n")

In [10]:
# Download the granules

GranuleHandler.download_from_granules(labsea2022_granules, session, path=str(filepath), threads=10)

Downloading:   0%|          | 0/71 [00:00<?, ?file/s]

[PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/AquaHkmLabSea2022b/MYD02HKM.A2022352.1640.061.2022354222030.hdf'),
 PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/AquaHkmLabSea2022b/MYD02HKM.A2022343.1650.061.2022344163236.hdf'),
 PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/AquaHkmLabSea2022b/MYD02HKM.A2022331.1630.061.2022332160811.hdf'),
 PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/AquaHkmLabSea2022b/MYD02HKM.A2022322.1640.061.2022324002759.hdf'),
 PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/AquaHkmLabSea2022b/MYD02HKM.A2022319.1610.061.2022320195116.hdf'),
 PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/AquaHkmLabSea2022b/MYD02HKM.A2022316.1540.061.2022317180202.hdf'),
 PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/AquaHkmLabSea2022b/MYD02HKM.A2022310.1620.061.2022311183559.hdf'),
 PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/AquaHkmLabSea2022b/MYD02HKM.A2022307.1550.061.2022308172954.hdf'),
 PosixPath('/home/eefjg/OneDrive